In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize


# -------------------------------------------------------
# Paramètres physiques
# -------------------------------------------------------
f = 40000          # fréquence (Hz)
c0 = 343           # vitesse du son dans l'air (m/s)
ω = 2 * np.pi * f
k = ω / c0         # nombre d'onde

# ---- Constantes physiques ----
k = 2 * np.pi * 40000 / 343
R_p = 0.0005
c_p = 2400
rho_p = 28.59
c_0 = 343
rho_0 = 1.204
R_t = 0.005  # rayon transducteur

#---------------------------------------------------------
eps = 1e-4  # pas de dérivation

omega = 2 * np.pi * 40000
k1 = 4 * np.pi/3 * R_p**3 / 4 * (1/(c_0**2*rho_0) - 1/(c_p**2*rho_p))
k2 = 3*4*np.pi/3*R_p**3/4 * ((rho_0 - rho_p)/(omega**2 * rho_0 * (rho_0 + 2*rho_p)))

# -------------------------------------------------------
# Espace de simulation
# -------------------------------------------------------
N =8 # nombre de transducteur par axe (8x8)
a=0.01


# bornes symétriques : +/-5 cm latéralement, 8 cm en hauteur
Lx = 0.05
Ly = 0.05
DzT=0.08

x = np.linspace(-Lx, Lx, N)
y = np.linspace(-Ly, Ly, N)

X_T_b, Y_T_b = np.meshgrid(x, y)
Z_T_b=np.array([[0]*N]*N)


##on passe tout à une dimension, dans le ref de la plaque :
X_T=X_T_b.flatten()
Y_T=Y_T_b.flatten()
Z_T=Z_T_b.flatten()

In [4]:
# === Fonctions utiles ===

def D_F_vectorized(xj, yj, zj, x, y, z):
    theta = np.arctan(np.sqrt((x - xj)**2 + (y - yj)**2) / np.where(np.abs(z - zj) < 1e-12, 1e-12, np.abs(z - zj)))
    df = np.sin(k * R_p * np.sin(theta)) / np.where(np.sin(k * R_p * np.sin(theta)) == 0, 1e-12, k * R_p * np.sin(theta))
    df = np.where(np.isnan(df), 1, df)
    return df

DJ={}
DF={}
def precompute_field_params(x, y, z):
    # Plaque du bas
    if (x,y,z) in DJ:
        return (DJ[(x,y,z)],DF[(x,y,z)])

    else :
      dx = X_T - x
      dy = Y_T - y
      dz = Z_T - z
      dj = np.sqrt(dx**2 + dy**2 + dz**2)
      df = D_F_vectorized(X_T, Y_T, Z_T, x, y, z)
      DJ[(x,y,z)]=dj
      DF[(x,y,z)]=df

      return (dj, df)

# === Calcul du champ total avec dj/df pré-calculés ===

def p_tot(phi_vals, x, y, z):
    dj, df = precompute_field_params(x, y, z)
    ptot = np.sum(df / dj * np.exp(1j * (k*dj + phi_vals)))
    return ptot

# === Gradient numérique ===

def grad_p(phi_vals, x, y, z):
    dp_dx = (p_tot(phi_vals, x+eps, y, z) - p_tot(phi_vals, x,y,z)) / eps
    dp_dy = (p_tot(phi_vals, x, y+eps, z) - p_tot(phi_vals, x,y,z)) / eps
    dp_dz = (p_tot(phi_vals, x, y, z+eps) - p_tot(phi_vals, x,y,z)) / eps
    return np.array([dp_dx, dp_dy, dp_dz])

# === Potentiel de Gorkov ===

def gorkov_potential(phi_vals, x, y, z):
    p_complex = p_tot(phi_vals, x,y,z)
    gradp = grad_p(phi_vals, x, y, z)

    return k1*abs(p_complex)**2 - k2*np.sum(np.abs(gradp)**2)

# === Laplacien du potentiel ===

def lap_U(phi_vals, x, y, z):
    U0 = gorkov_potential(phi_vals, x, y, z)
    Ux = gorkov_potential(phi_vals, x+eps, y, z)
    Uy = gorkov_potential(phi_vals, x, y+eps, z)
    Uz = gorkov_potential(phi_vals, x, y, z+eps)
    return (Ux - 2*U0 + gorkov_potential(phi_vals, x-eps, y, z))/eps**2 + \
           (Uy - 2*U0 + gorkov_potential(phi_vals, x, y-eps, z))/eps**2 + \
           (Uz - 2*U0 + gorkov_potential(phi_vals, x, y, z-eps))/eps**2

# === Fonction objectif pour l’optimisation ===

def F_opt(phi_vals, xf1, yf1, zf1,xf2, yf2, zf2):
    ptot1 = p_tot(phi_vals, xf1, yf1, zf1)
    lapU1 = lap_U(phi_vals, xf1, yf1, zf1)
    ptot2=p_tot(phi_vals, xf2, yf2, zf2)
    lapU2=lap_U(phi_vals, xf2, yf2, zf2)
    ptot=ptot1+ptot2
    lapU=lapU1+lapU2
    return abs(ptot) - lapU

# === optimisation ===

def optimizer(xf1, yf1, zf1,xf2, yf2, zf2):
    def objective(phi_vals):
        phi_mod = np.mod(phi_vals, 2*np.pi)
        return F_opt(phi_mod, xf1, yf1, zf1, xf2, yf2, zf2)

    phi0 = np.zeros((N**2))
    result = minimize(objective, phi0, method='L-BFGS-B')
    phases_mod = np.mod(result.x, 2*np.pi)

    print(", ".join(f"{a:.4f}" for a in phases_mod))
    return phases_mod

In [6]:
# === Point de focalisation ===
X_foc1, Y_foc1, Z_foc1 = 0.01, 0, 0.03
X_foc2, Y_foc2, Z_foc2 = -0.01, 0, 0.03

# === Lancement ===
optimizer(X_foc1, Y_foc1, Z_foc1, X_foc2, Y_foc2, Z_foc2)

6.2371, 5.9838, 0.6663, 0.3726, 0.3726, 0.6663, 5.9838, 6.2371, 0.0767, 5.9588, 6.1535, 0.0958, 0.0958, 6.1535, 5.9588, 0.0767, 5.3361, 6.1957, 5.9844, 6.0822, 6.0822, 5.9844, 6.1957, 5.3361, 0.7418, 6.2090, 0.4455, 0.0096, 0.0096, 0.4455, 6.2090, 0.7418, 0.7418, 6.2090, 0.4455, 0.0096, 0.0096, 0.4455, 6.2090, 0.7418, 5.3361, 6.1957, 5.9844, 6.0822, 6.0822, 5.9844, 6.1957, 5.3361, 0.0767, 5.9588, 6.1535, 0.0958, 0.0958, 6.1535, 5.9588, 0.0767, 6.2371, 5.9838, 0.6663, 0.3726, 0.3726, 0.6663, 5.9838, 6.2371


array([6.23710166, 5.98377587, 0.66627968, 0.37264448, 0.37264446,
       0.66627965, 5.98377588, 6.23710166, 0.07666009, 5.95883064,
       6.15345989, 0.0957528 , 0.09575279, 6.15345993, 5.95883059,
       0.07666014, 5.33611381, 6.195653  , 5.98444184, 6.08220652,
       6.0822066 , 5.98444195, 6.19565292, 5.3361137 , 0.74177686,
       6.20895272, 0.44545657, 0.00956136, 0.00956134, 0.44545655,
       6.20895263, 0.74177688, 0.74177686, 6.20895263, 0.44545653,
       0.00956136, 0.00956134, 0.44545655, 6.20895264, 0.74177688,
       5.33611381, 6.19565301, 5.98444197, 6.08220648, 6.08220652,
       5.98444186, 6.19565291, 5.33611383, 0.07666021, 5.95883058,
       6.15345973, 0.0957528 , 0.0957528 , 6.15345994, 5.95883058,
       0.07666021, 6.23710161, 5.98377574, 0.66627968, 0.37264445,
       0.37264452, 0.66627966, 5.98377587, 6.23710161])